# Chapter 13: TensorFlow

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",  one_hot=True)

## simple softmax model

In [16]:
import tensorflow as tf

# creat the model
x = tf.placeholder(tf.float32, [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.matmul(x, W) + b

In [17]:
#define  loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), 
#                                              reduction_indices=[1]))
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)


In [18]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

#train the model
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})

In [19]:
#test the model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9195


## deep neural network

In [24]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# initialize weight
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# initialize bias
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [28]:
# convlution with filters
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

# max pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding='SAME')

# how to understand the strides

In [29]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# Reshape to use within a convolutional neural net.
# Last dimension is for "features" - there is only one here, since images are
# grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.

x_image = tf.reshape(x, [-1, 28, 28, 1])

# conv Net 1
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [30]:
# conv Net 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [31]:
# fully connected 1
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [33]:
# drop out
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [34]:
# fc 2
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [39]:
cross_entropy = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, training accuracy %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1],  keep_prob:0.5})
print("test accuracy %g" %(accuracy.eval(feed_dict={
    x: mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})))
        

step 0, training accuracy 0.08
step 100, training accuracy 0.88
step 200, training accuracy 0.94
step 300, training accuracy 0.88
step 400, training accuracy 0.9
step 500, training accuracy 0.86
step 600, training accuracy 0.94
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.98
step 1000, training accuracy 0.94
step 1100, training accuracy 0.94
step 1200, training accuracy 0.9
step 1300, training accuracy 0.98
step 1400, training accuracy 0.94
step 1500, training accuracy 0.98
step 1600, training accuracy 1
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
test accuracy 0.9769


In [64]:
# all size of variables

#dir(mnist.train.images)
# print(mnist.train.next_batch(1)[0].shape)
# x.shape -> (batch_size, 784)
# W.shape -> (784, 10)
# b.shape -> (10)
# y.shape /  y_.shape -> (batch_size, 10)
# x_image -> (batch_size, 28, 28, 1)
# h_conv1 -> (batch_size, 28, 28, 32)
# 之前一直模糊没搞清楚的地方，conv需要知道图片的color channel，RGB是3，x_image最后一维应该
# 是3. 而且filter的size应该是FxFxcolor channel，而且每个filter会和image的各个color channel
# 做内积然后相加，就是3个channel得到一个数。所以h_conv1最后一维是32，就是filter的个数
# filter是4维tensor，这个不知道之前有没有注意到
# h_pool1 -> (batch_size, 14, 14, 32)
# h_conv2 -> (batch_size, 14, 14, 64)
# h_pool2 -> (batch_size, 7, 7, 64)
# h_pool2_flat -> (batch_size, 7*7*64)
# h_fc1 -> (batch_size, 1024)
# h_fc1_drop -> (batch_size, 1024)
# y_conv -> (batch_size, 10)


(1, 784)
